In [1]:
#구 비율 데이터 만들기 -> sig_info_weight만들기

In [26]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [48]:
code = pd.read_csv('../Data/processing_data/2nd/busan_code.csv')

In [49]:
sig_code = code[code['DONG'].isna()].reset_index(drop=True)

In [50]:
sig_code = sig_code.drop(columns='DONG')

In [51]:
#펌프작업
pump_data1 = pd.read_csv('../data/rawdata/busan_pump_data.csv', encoding='euc-kr')
pump_data2 = gpd.read_file('../Data/rawdata/FL_PUMP/FL_PUMP_SP.shp')
pump_data1.iloc[29:31,0:1] = '금정구'
pump_data1.iloc[26:27,0:1] = '연제구'
code['CODE'] = code['CODE'].apply(str)
code
pump_data2 = pd.merge(code, pump_data2, left_on='CODE', right_on='ADM_CD', how='inner')
pump_data2.shape
pump_data2 = pump_data2.groupby(by='ZONE').count()['CODE']
pump_data2.sum()
pump_data1 = pump_data1.groupby(by='구군').count()['배수펌프장명']
pump_data = pump_data1.add(pump_data2,fill_value=0)
pump_data = pump_data.reset_index()
pump_data.rename(columns={'index':'ZONE',0:'PUMP_COUNT'},inplace=True)
pump_data['PUMP_COUNT'] = pump_data['PUMP_COUNT'].astype(int)
PIH_merge = pd.merge(pump_data,sig_code,on=['ZONE'],how='inner')

In [52]:
#불투수면 작업

In [53]:
PIH_merge

,ZONE,PUMP_COUNT,CODE
0,강서구,22,2644000000
1,금정구,2,2641000000
2,기장군,3,2671000000
3,남구,7,2629000000
4,동구,2,2617000000
5,동래구,4,2626000000
6,북구,6,2632000000
7,사상구,14,2653000000
8,사하구,7,2638000000
9,연제구,5,2647000000


In [54]:
#전국 데이터에서 부산 데이터를 뽑아냄 불투수면의 경우 법정동코드로 되어 있고 
# 부산의 법정동코드는 21로 시작하는데 부산의 '구' 데이터만 가져온다.
#busan_code = busan_code.rename(columns={'법정동코드':'CODE','n2':'ZONE','n3':'DONG'})

In [55]:
imper = pd.read_excel('../data/rawdata/imper_ratio_data.xlsx')
imper['코드'] = imper['코드'].apply(str)
imper = imper[imper['코드'].str.startswith('21')].reset_index(drop=True)
imper = imper.drop(index=0).reset_index(drop=True)
imper = imper.drop(columns=['코드','기준연도','불투수면 면적(㎢)'])
imper.rename(columns={'시도시군구':'ZONE', '불투수면 비율(%)':'IMP_SUR_RATIO',\
                              '행정구역면적(㎢)':'ZONE_AREA'}, inplace=True)
imper['ZONE_AREA'] = round(imper['ZONE_AREA'],2) * 10000000
imper['IMP_SUR_RATIO'] = round(imper['IMP_SUR_RATIO'],2)

imper.sort_values('ZONE',inplace=True, ascending=False, ignore_index=True)


In [56]:
PIH_merge = pd.merge(PIH_merge,imper, on='ZONE', how='outer')

In [57]:
PIH_merge

,ZONE,PUMP_COUNT,CODE,IMP_SUR_RATIO,ZONE_AREA
0,강서구,22.0,2.644000e+09,24.36,1.848800e+09
1,금정구,2.0,2.641000e+09,18.94,6.573000e+08
2,기장군,3.0,2.671000e+09,7.70,2.198800e+09
3,남구,7.0,2.629000e+09,51.82,2.728000e+08
4,동구,2.0,2.617000e+09,57.92,1.009000e+08
5,동래구,4.0,2.626000e+09,59.03,1.667000e+08
6,북구,6.0,2.632000e+09,22.17,3.976000e+08
7,사상구,14.0,2.653000e+09,40.23,3.625000e+08
8,사하구,7.0,2.638000e+09,38.20,4.711000e+08
9,연제구,5.0,2.647000e+09,62.32,1.215000e+08


In [58]:
PIH_merge.fillna(0,inplace=True)
PIH_merge['PUMP_RATIO'] = PIH_merge['PUMP_COUNT'] * 10 / PIH_merge['ZONE_AREA']
PIH_merge.drop(columns='PUMP_COUNT', inplace=True)

In [59]:
#맨홀작업

# manhole = pd.read_excel("../Data/rawdata/manhole_data.xlsx", sheet_name="2-1_지역별 현황", header = 1)

# manhole = manhole[manhole["행정구역"] == '부산광역시'][['Unnamed: 2', '합류식맨홀\n(개소)']].iloc[2:]

# manhole.rename(columns={'Unnamed: 2':'ZONE', '합류식맨홀\n(개소)':'MANHOLES_COUNT'}, inplace=True)
# manhole.reset_index(drop=True)
# manhole['MANHOLES_COUNT'] = manhole['MANHOLES_COUNT'].astype(int)

# PIH_merge = pd.merge(PIH_merge, manhole, on='ZONE', how='left')

# PIH_merge

In [60]:
PIH_merge

,ZONE,CODE,IMP_SUR_RATIO,ZONE_AREA,PUMP_RATIO
0,강서구,2.644000e+09,24.36,1.848800e+09,1.189961e-07
1,금정구,2.641000e+09,18.94,6.573000e+08,3.042751e-08
2,기장군,2.671000e+09,7.70,2.198800e+09,1.364381e-08
3,남구,2.629000e+09,51.82,2.728000e+08,2.565982e-07
4,동구,2.617000e+09,57.92,1.009000e+08,1.982161e-07
5,동래구,2.626000e+09,59.03,1.667000e+08,2.399520e-07
6,북구,2.632000e+09,22.17,3.976000e+08,1.509054e-07
7,사상구,2.653000e+09,40.23,3.625000e+08,3.862069e-07
8,사하구,2.638000e+09,38.20,4.711000e+08,1.485884e-07
9,연제구,2.647000e+09,62.32,1.215000e+08,4.115226e-07


In [61]:
#침수빈도

In [74]:
df = pd.read_csv("../Data/processing_data/1st/busan_flood_geo.csv")
df1 = df.copy()
a = df1[["ZONE","F_BEGIN_DE"]]
b = a.copy()
b.drop_duplicates(["ZONE","F_BEGIN_DE"],inplace=True)
b.astype({"ZONE":str})

# 가중치 구하기
zone = ['연제구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구',
       '금정구', '강서구', '수영구', '사상구', '기장군']
count = [4,1,1,2,1,1,1,1,3,5,1,3,4,1,1]
w = pd.DataFrame({"ZONE":zone,"COUNT":count})
bins = pd.DataFrame({"bin":w.groupby("COUNT").size()})
bins["percentage"]=round((bins["bin"]/bins["bin"].sum())*100,2)
bins["COUNT1"]=[1,2,3,4,5]
F_bin = pd.merge(w,bins,how="inner",left_on="COUNT",right_on="COUNT1")
F_bin.drop(["COUNT1","bin"],axis=1,inplace=True)
F_bin.rename(columns={"COUNT":"F_GRADE","percentage":"F_PERCENTAGE"},inplace=True)

In [82]:
F_bin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ZONE          15 non-null     object 
 1   F_GRADE       15 non-null     int64  
 2   F_PERCENTAGE  15 non-null     float64
 3   F_WEIGHT      15 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 600.0+ bytes


In [84]:
F_bin
F_bin["F_WEIGHT"] = float(1.0)/F_bin["F_PERCENTAGE"]
F_bin

,ZONE,F_GRADE,F_PERCENTAGE,F_WEIGHT
0,연제구,4,13.33,0.075019
1,수영구,4,13.33,0.075019
2,서구,1,60.00,0.016667
3,동구,1,60.00,0.016667
4,부산진구,1,60.00,0.016667
5,동래구,1,60.00,0.016667
6,남구,1,60.00,0.016667
7,북구,1,60.00,0.016667
8,금정구,1,60.00,0.016667
9,사상구,1,60.00,0.016667


In [78]:
F_bin.head(1)

,ZONE,F_GRADE,F_PERCENTAGE,F_WEIGHT
0,연제구,4,13.33,0.075019


In [63]:
PIH_merge = pd.merge(PIH_merge, F_bin, on='ZONE', how='left')
PIH_merge = PIH_merge.fillna(0)

In [64]:
PIH_merge = PIH_merge.drop(columns='CODE')
PIH_merge = pd.merge(PIH_merge, sig_code, on='ZONE', how='left')

In [65]:
PIH_merge

,ZONE,IMP_SUR_RATIO,ZONE_AREA,PUMP_RATIO,F_GRADE,F_PERCENTAGE,F_WEIGHT,CODE
0,강서구,24.36,1.848800e+09,1.189961e-07,3.0,13.33,0.075019,2644000000
1,금정구,18.94,6.573000e+08,3.042751e-08,1.0,60.00,0.016667,2641000000
2,기장군,7.70,2.198800e+09,1.364381e-08,1.0,60.00,0.016667,2671000000
3,남구,51.82,2.728000e+08,2.565982e-07,1.0,60.00,0.016667,2629000000
4,동구,57.92,1.009000e+08,1.982161e-07,1.0,60.00,0.016667,2617000000
5,동래구,59.03,1.667000e+08,2.399520e-07,1.0,60.00,0.016667,2626000000
6,북구,22.17,3.976000e+08,1.509054e-07,1.0,60.00,0.016667,2632000000
7,사상구,40.23,3.625000e+08,3.862069e-07,1.0,60.00,0.016667,2653000000
8,사하구,38.20,4.711000e+08,1.485884e-07,5.0,6.67,0.149925,2638000000
9,연제구,62.32,1.215000e+08,4.115226e-07,4.0,13.33,0.075019,2647000000


In [66]:
PIH_merge.to_csv('../Data/processing_data/2nd/sig_info.csv',encoding='utf-8', index=False)

In [67]:
# 불투수면 가중치 계산

In [68]:
imp_weight = PIH_merge['IMP_SUR_RATIO'].copy()

In [69]:
for idx,col in enumerate(PIH_merge['IMP_SUR_RATIO']):
    if col <= 15:
        imp_weight[idx] = col / 6.66
    elif col > 15 and col <= 25:
        imp_weight[idx] = col / (20 * 2)
    elif col > 25 and col <= 40:
        imp_weight[idx] = col / (13.33 * 3)
    elif col > 40 and col <= 55:
        imp_weight[idx] = col / (33.33 * 4)
    elif col > 55:
        imp_weight[idx] = col / (26.67 * 5)

In [70]:
PIH_merge['Impervious_Surface_Weight'] = imp_weight

In [71]:
PIH_merge

,ZONE,IMP_SUR_RATIO,ZONE_AREA,PUMP_RATIO,F_GRADE,F_PERCENTAGE,F_WEIGHT,CODE,Impervious_Surface_Weight
0,강서구,24.36,1.848800e+09,1.189961e-07,3.0,13.33,0.075019,2644000000,0.609000
1,금정구,18.94,6.573000e+08,3.042751e-08,1.0,60.00,0.016667,2641000000,0.473500
2,기장군,7.70,2.198800e+09,1.364381e-08,1.0,60.00,0.016667,2671000000,1.156156
3,남구,51.82,2.728000e+08,2.565982e-07,1.0,60.00,0.016667,2629000000,0.388689
4,동구,57.92,1.009000e+08,1.982161e-07,1.0,60.00,0.016667,2617000000,0.434346
5,동래구,59.03,1.667000e+08,2.399520e-07,1.0,60.00,0.016667,2626000000,0.442670
6,북구,22.17,3.976000e+08,1.509054e-07,1.0,60.00,0.016667,2632000000,0.554250
7,사상구,40.23,3.625000e+08,3.862069e-07,1.0,60.00,0.016667,2653000000,0.301755
8,사하구,38.20,4.711000e+08,1.485884e-07,5.0,6.67,0.149925,2638000000,0.955239
9,연제구,62.32,1.215000e+08,4.115226e-07,4.0,13.33,0.075019,2647000000,0.467342


In [72]:
PIH_merge = PIH_merge[['CODE','ZONE','PUMP_RATIO','F_WEIGHT','Impervious_Surface_Weight' ]]

In [73]:
PIH_merge.to_csv('../Data/processing_data/2nd/sig_info_weight.csv', encoding='utf-8', index=False)